In [39]:
# 1. IMPORT LIBRARIES
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [40]:
import sqlite3

conn = sqlite3.connect('Cantones.db')
cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print("Tables in Cantones.db:", tables)


Tables in Cantones.db: [('hogar_cant',), ('poblacion_cant',), ('vivienda_cant',), ('df_renamed_table',), ('df_renamed_hogar',)]


In [38]:
import sqlite3 as sqlite3

conn = sqlite3.connect('Cantones.db')

df_hogar_renamed = pd.read_sql('SELECT * FROM df_renamed_hogar', conn)
df_vivienda_renamed = pd.read_sql('SELECT * FROM df_renamed_table', conn)


KeyboardInterrupt: 

Combinamos ambas tablas:

In [19]:
df_merged = pd.merge(
    df_vivienda_renamed,
    df_hogar_renamed,
    on=['provincia','canton_id','id_vivienda','id_hogar'],  # adjust if needed
    how='inner'
)

print("\nMerged DF shape:", df_merged.shape)
print(df_merged.head())



Merged DF shape: (2381224, 84)
   provincia  canton_id  nro_vivienda_x  nro_hogar_x  nro_dormitorios_x  \
0          5          1             1.0            1                1.0   
1          5          1             2.0            1                4.0   
2          5          1             3.0            1                3.0   
3          5          1             4.0            1                3.0   
4          5          1             5.0            1                2.0   

   espacio_cocina_x  tiene_inodoro_x  tiene_ducha_x  combustible_cocina_x  \
0               2.0              1.0            1.0                   1.0   
1               1.0              1.0            3.0                   1.0   
2               1.0              1.0            1.0                   1.0   
3               2.0              1.0            3.0                   1.0   
4               1.0              1.0            1.0                   4.0   

   fuente_agua_bebida_x  ...  emigrantes_desde_2010_y 

In [20]:
#We take a look at the missing values
missing_counts = df_merged.isnull().sum().sort_values(ascending=False)
print("\nMissing values by column:\n", missing_counts)


Missing values by column:
 nro_emigrantes_x       2340868
nro_emigrantes_y       2340868
nro_fallecidos_x       2276356
nro_fallecidos_y       2276356
nro_gatos_x            1758342
                        ...   
total_mujeres_x              0
total_hombres_x              0
nro_hogar_x                  0
nro_vivienda_x               0
registro_imputado_y          0
Length: 84, dtype: int64


In [24]:
print("Columnas actuales:")
print(df_merged.columns.tolist())

Columnas actuales:
['provincia', 'canton_id', 'nro_vivienda_x', 'nro_hogar_x', 'nro_dormitorios_x', 'espacio_cocina_x', 'tiene_inodoro_x', 'tiene_ducha_x', 'combustible_cocina_x', 'fuente_agua_bebida_x', 'separa_basura_organica_x', 'separa_basura_animales_x', 'separa_basura_reciclaje_x', 'tiene_perros_x', 'nro_perros_x', 'tiene_gatos_x', 'nro_gatos_x', 'tenencia_vivienda_x', 'tiene_telefono_fijo_x', 'tiene_celular_x', 'tiene_tv_pagada_x', 'tiene_internet_x', 'tiene_computadora_x', 'tiene_refrigeradora_x', 'tiene_lavadora_x', 'tiene_secadora_x', 'tiene_microondas_x', 'tiene_extractora_olores_x', 'tiene_auto_x', 'tiene_moto_x', 'fallecidos_ultimos_3_anios_x', 'nro_fallecidos_x', 'emigrantes_desde_2010_x', 'nro_emigrantes_x', 'total_hombres_x', 'total_mujeres_x', 'total_personas_x', 'persona_no_mencionada_x', 'zona_x', 'canton_x', 'id_vivienda', 'id_hogar', 'nro_dormitorios_r_x', 'registro_imputado_x', 'nro_vivienda_y', 'nro_hogar_y', 'nro_dormitorios_y', 'espacio_cocina_y', 'tiene_inodor

In [25]:
# Identify columns ending in _x and _y
x_cols = [col for col in df_merged.columns if col.endswith('_x')]
y_cols = [col for col in df_merged.columns if col.endswith('_y')]

# For each _x column, check if there's a corresponding _y column,
# and compare them row by row
for x_col in x_cols:
    # Get the base name by removing '_x' at the end
    base_name = x_col[:-2]  
    y_col = base_name + '_y'
    
    if y_col in y_cols:
        # Check if ALL rows are identical
        are_identical = (df_merged[x_col] == df_merged[y_col]).all()
        
        # Print the result
        print(f"Comparing {x_col} vs {y_col}:")
        print(" → Identical across all rows?", are_identical)
        


Comparing nro_vivienda_x vs nro_vivienda_y:
 → Identical across all rows? True
Comparing nro_hogar_x vs nro_hogar_y:
 → Identical across all rows? True
Comparing nro_dormitorios_x vs nro_dormitorios_y:
 → Identical across all rows? False
Comparing espacio_cocina_x vs espacio_cocina_y:
 → Identical across all rows? False
Comparing tiene_inodoro_x vs tiene_inodoro_y:
 → Identical across all rows? False
Comparing tiene_ducha_x vs tiene_ducha_y:
 → Identical across all rows? False
Comparing combustible_cocina_x vs combustible_cocina_y:
 → Identical across all rows? False
Comparing fuente_agua_bebida_x vs fuente_agua_bebida_y:
 → Identical across all rows? False
Comparing separa_basura_organica_x vs separa_basura_organica_y:
 → Identical across all rows? False
Comparing separa_basura_animales_x vs separa_basura_animales_y:
 → Identical across all rows? False
Comparing separa_basura_reciclaje_x vs separa_basura_reciclaje_y:
 → Identical across all rows? False
Comparing tiene_perros_x vs tien

In [26]:
# Check for entire row duplicates
duplicates = df_merged.duplicated()

# How many duplicated rows?
num_duplicates = duplicates.sum()
print("Number of fully duplicated rows:", num_duplicates)

# (Optional) Look at the first few duplicates
if num_duplicates > 0:
    print("Sample duplicated rows:")
    display(df_merged[duplicates].head())


Number of fully duplicated rows: 0


CONCLUSIONES: 
- Al combinar las tablas tenemos columnas con informacion repetida, esto se pudo comprobar gracias a la comparación fila por fila, además, al mezclar las columnas se agregó el sufijo x y y, esto fue removido. 
- No hay columnas completamente duplicadas

In [27]:
# 1. Identify columns ending in _x
x_cols = [col for col in df_merged.columns if col.endswith('_x')]

# 2. Loop through each _x column, find its _y counterpart, compare, drop if identical
for x_col in x_cols:
    # Derive the matching _y column name
    y_col = x_col.replace('_x', '_y')
    
    # Only proceed if _y column actually exists
    if y_col in df_merged.columns:
        # Compare the entire Series to check if they're identical
        # .equals() returns True if every element matches, incl. NaNs
        if df_merged[x_col].equals(df_merged[y_col]):
            df_merged.drop(columns=[x_col], inplace=True)

print("Columns after dropping identical _x columns:")
print(df_merged.columns.tolist())


Columns after dropping identical _x columns:
['provincia', 'canton_id', 'id_vivienda', 'id_hogar', 'nro_vivienda_y', 'nro_hogar_y', 'nro_dormitorios_y', 'espacio_cocina_y', 'tiene_inodoro_y', 'tiene_ducha_y', 'combustible_cocina_y', 'fuente_agua_bebida_y', 'separa_basura_organica_y', 'separa_basura_animales_y', 'separa_basura_reciclaje_y', 'tiene_perros_y', 'nro_perros_y', 'tiene_gatos_y', 'nro_gatos_y', 'tenencia_vivienda_y', 'tiene_telefono_fijo_y', 'tiene_celular_y', 'tiene_tv_pagada_y', 'tiene_internet_y', 'tiene_computadora_y', 'tiene_refrigeradora_y', 'tiene_lavadora_y', 'tiene_secadora_y', 'tiene_microondas_y', 'tiene_extractora_olores_y', 'tiene_auto_y', 'tiene_moto_y', 'fallecidos_ultimos_3_anios_y', 'nro_fallecidos_y', 'emigrantes_desde_2010_y', 'nro_emigrantes_y', 'total_hombres_y', 'total_mujeres_y', 'total_personas_y', 'persona_no_mencionada_y', 'zona_y', 'canton_y', 'nro_dormitorios_r_y', 'registro_imputado_y']


Con el paso anterior, sacamos las columnas idénticas para sólo mantener una. 

In [28]:
columns_to_drop = [
    # Identificación redundante (si ya tienes id_vivienda / id_hogar o sufijo _x)
    'nro_vivienda_y', 
    'nro_hogar_y',
    
    # Manejo de residuos
    'separa_basura_organica_y', 'separa_basura_animales_y', 'separa_basura_reciclaje_y',
    
    # Equipamiento muy específico (poco aporte socioeconómico comparado con otras col.)
    'tiene_tv_pagada_y', 'tiene_microondas_y', 'tiene_extractora_olores_y', 'tiene_secadora_y',
    
    # Otras columnas con menor impacto en predicción
    'persona_no_mencionada_y',
    'registro_imputado_y'
]


In [29]:
df_merged.drop(columns=columns_to_drop, inplace=True, errors='ignore')


Definimos vulnerabilidad estructural combinando indicadores

In [35]:
print("Columnas actuales:")
print(df_merged.columns.tolist())
print(df_merged.dtypes)

Columnas actuales:
['provincia', 'canton_id', 'id_vivienda', 'id_hogar', 'nro_dormitorios_y', 'espacio_cocina_y', 'tiene_inodoro_y', 'tiene_ducha_y', 'combustible_cocina_y', 'fuente_agua_bebida_y', 'tiene_perros_y', 'nro_perros_y', 'tiene_gatos_y', 'nro_gatos_y', 'tenencia_vivienda_y', 'tiene_telefono_fijo_y', 'tiene_celular_y', 'tiene_internet_y', 'tiene_computadora_y', 'tiene_refrigeradora_y', 'tiene_lavadora_y', 'tiene_auto_y', 'tiene_moto_y', 'fallecidos_ultimos_3_anios_y', 'nro_fallecidos_y', 'emigrantes_desde_2010_y', 'nro_emigrantes_y', 'total_hombres_y', 'total_mujeres_y', 'total_personas_y', 'zona_y', 'canton_y', 'nro_dormitorios_r_y']
provincia                         int64
canton_id                         int64
id_vivienda                     float64
id_hogar                         object
nro_dormitorios_y               float64
espacio_cocina_y                float64
tiene_inodoro_y                 float64
tiene_ducha_y                   float64
combustible_cocina_y       

In [ ]:
categorical_cols = [
    'espacio_cocina_y',
    'tiene_inodoro_y',
    'tiene_ducha_y',
    'combustible_cocina_y',
    'fuente_agua_bebida_y',
    'tiene_perros_y',
    'tiene_gatos_y',
    'tenencia_vivienda_y',
    'tiene_telefono_fijo_y',
    'tiene_celular_y',
    'tiene_internet_y',
    'tiene_computadora_y',
    'tiene_refrigeradora_y',
    'tiene_lavadora_y',
    'tiene_auto_y',
    'tiene_moto_y'
    # ... Agrega otras que sean códigos sin orden numérico real
]

# 1. Conviértelas a string para que get_dummies las trate como categorías
df[categorical_cols] = df[categorical_cols].astype('Int64').astype(str)

# 2. Aplica One-Hot Encoding
df = pd.get_dummies(df, columns=categorical_cols, prefix=categorical_cols)

# Observa el resultado
print(df.head())

# Revisa las nuevas columnas creadas
new_cols = [col for col in df.columns if any(prefix in col for prefix in categorical_cols)]
print("\nNuevas columnas dummies creadas:")
print(new_cols)
